In [1]:
from pyspark.sql import functions as F 
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel

In [2]:
list=[[['url1','url2','url3','url4','url5','url6','url7','url8','url9','url10','url11','url12'],
        ['adlink','picture','link','link','link','link','picture','link','link','link','link','adlink']]]

df=spark.createDataFrame(list,['urls','type'])
df.show(truncate=False)

+---------------------------------------------------------------------------+----------------------------------------------------------------------------------+
urls |type |
+---------------------------------------------------------------------------+----------------------------------------------------------------------------------+
[url1, url2, url3, url4, url5, url6, url7, url8, url9, url10, url11, url12]|[adlink, picture, link, link, link, link, picture, link, link, link, link, adlink]|
+---------------------------------------------------------------------------+----------------------------------------------------------------------------------+

In [3]:
df.withColumn("type", F.expr("""transform(type, x-> IF(x=='adlink' or x='picture',0,x))"""))\
  .withColumn("type_urls", F.arrays_zip(F.col("urls"),F.col("type"))).select("type_urls")\
  .withColumn("urls1", F.expr("""filter(type_urls, x-> x.type!='0')"""))\
  .select(F.col("urls1.urls"), F.col("urls1.type")).show(truncate=False)



+--------------------------------------------------+------------------------------------------------+
urls |type |
+--------------------------------------------------+------------------------------------------------+
[url3, url4, url5, url6, url8, url9, url10, url11]|[link, link, link, link, link, link, link, link]|
+--------------------------------------------------+------------------------------------------------+

In [4]:
df.withColumn("type_urls", F.arrays_zip(F.col("urls"),F.col("type"))).select("type_urls")\
  .withColumn("urls1", F.expr("""filter(type_urls, x-> x.type!='adlink' and x.type!='picture')"""))\
  .select(F.col("urls1.urls"), F.col("urls1.type")).show(truncate=False)

+--------------------------------------------------+------------------------------------------------+
urls |type |
+--------------------------------------------------+------------------------------------------------+
[url3, url4, url5, url6, url8, url9, url10, url11]|[link, link, link, link, link, link, link, link]|
+--------------------------------------------------+------------------------------------------------+

In [5]:
df.withColumn("type_urls", F.arrays_zip(F.col("urls"),F.col("type"))).select("type_urls")\
  .withColumn("urls1", F.expr("""filter(type_urls, x-> x.type!='adlink' and x.type!='picture')"""))\
  .select(F.col("urls1.urls"), F.col("urls1.type")).show(truncate=False)

+--------------------------------------------------+------------------------------------------------+
urls |type |
+--------------------------------------------------+------------------------------------------------+
[url3, url4, url5, url6, url8, url9, url10, url11]|[link, link, link, link, link, link, link, link]|
+--------------------------------------------------+------------------------------------------------+

In [6]:
parameter=[6,9,8,1,4,6,9,10,2,11]

list=[[-4,1.00],
     [0,8.2],
     [1,8.2],
     [2,9.19],
     [3,5.2],
     [4,3.2],
     [5,2.9],
     [6,9.11],
     [7,1.21]]
df=spark.createDataFrame(list,['index','feature'])
df.show()

+-----+-------+
index|feature|
+-----+-------+
 -4| 1.0|
 0| 8.2|
 1| 8.2|
 2| 9.19|
 3| 5.2|
 4| 3.2|
 5| 2.9|
 6| 9.11|
 7| 1.21|
+-----+-------+

In [7]:
from pyspark.sql import functions as F
parameter=[6,9,8,1,4,6,9,10,2,11]
df.withColumn("parameter", F.array(*[F.lit(x) for x in parameter]))\
  .withColumn("multiplied", F.when(F.col("index")<=F.lit(-1),\
                                   F.expr("""element_at(parameter,int(index)) * feature"""))\
              .otherwise(F.expr("""element_at(parameter,int(index+1)) * feature""")))\
  .show(truncate=False)

+-----+-------+--------------------------------+------------------+
index|feature|parameter |multiplied |
+-----+-------+--------------------------------+------------------+
-4 |1.0 |[6, 9, 8, 1, 4, 6, 9, 10, 2, 11]|9.0 |
0 |8.2 |[6, 9, 8, 1, 4, 6, 9, 10, 2, 11]|49.199999999999996|
1 |8.2 |[6, 9, 8, 1, 4, 6, 9, 10, 2, 11]|73.8 |
2 |9.19 |[6, 9, 8, 1, 4, 6, 9, 10, 2, 11]|73.52 |
3 |5.2 |[6, 9, 8, 1, 4, 6, 9, 10, 2, 11]|5.2 |
4 |3.2 |[6, 9, 8, 1, 4, 6, 9, 10, 2, 11]|12.8 |
5 |2.9 |[6, 9, 8, 1, 4, 6, 9, 10, 2, 11]|17.4 |
6 |9.11 |[6, 9, 8, 1, 4, 6, 9, 10, 2, 11]|81.99 |
7 |1.21 |[6, 9, 8, 1, 4, 6, 9, 10, 2, 11]|12.1 |
+-----+-------+--------------------------------+------------------+

In [8]:
int(-1)

Out[19]: -1

In [9]:
import pandas as pd
import datetime

data = {'date': ['2014-01-01', '2014-01-02', '2014-01-03', '2014-01-04', '2014-01-05', '2014-01-06'],
     'customerid': [1, 2, 2, 3, 4, 3], 'productids': ['A;B', 'D;E', 'G;X', 'P;Q', 'S;T;U;G', 'U;G;G;G']}
data = pd.DataFrame(data)
data['date'] = pd.to_datetime(data['date'])

In [10]:
df=spark.createDataFrame(data)
w=Window().partitionBy("customerid").orderBy(F.col("date").cast("long")).rangeBetween(-86400*2,0)
df.withColumn("productids", F.split("productids", "\;"))\
  .withColumn("products", F.flatten(F.collect_list("productids").over(w)))\
  .withColumn("duplic", F.size("productids")-F.size(F.array_distinct("productids")))\
  .withColumn("result", (F.size("products") - F.size(F.array_distinct("products")))-F.col("duplic"))\
  .orderBy(F.col("date")).drop("productids","products","duplic").show()

+-------------------+----------+------+
 date|customerid|result|
+-------------------+----------+------+
2014-01-01 00:00:00| 1| 0|
2014-01-02 00:00:00| 2| 0|
2014-01-03 00:00:00| 2| 0|
2014-01-04 00:00:00| 3| 0|
2014-01-05 00:00:00| 4| 0|
2014-01-06 00:00:00| 3| 2|
+-------------------+----------+------+

In [11]:
df=spark.createDataFrame(data)
df.show()
from pyspark.sql import functions as F
from pyspark.sql.window import Window
w=Window().partitionBy("customerid").orderBy(F.col("date").cast("long")).rangeBetween(-86400*2,0)
df.withColumn("productids", F.array_distinct(F.split("productids", "\;")))\
  .withColumn("products", F.flatten((F.collect_list("productids").over(w))))\
  .withColumn("result", F.when(F.size("products")!=F.size(F.array_distinct("products")),F.lit(1)).otherwise(F.lit(0)))\
  .drop("productids","products").orderBy("date").show()

+-------------------+----------+----------+
 date|customerid|productids|
+-------------------+----------+----------+
2014-01-01 00:00:00| 1| A;B|
2014-01-02 00:00:00| 2| D;E|
2014-01-03 00:00:00| 2| G;X|
2014-01-04 00:00:00| 3| P;Q|
2014-01-05 00:00:00| 4| S;T;U;G|
2014-01-06 00:00:00| 3| U;G;G;G|
+-------------------+----------+----------+

+-------------------+----------+------+
 date|customerid|result|
+-------------------+----------+------+
2014-01-01 00:00:00| 1| 0|
2014-01-02 00:00:00| 2| 0|
2014-01-03 00:00:00| 2| 0|
2014-01-04 00:00:00| 3| 0|
2014-01-05 00:00:00| 4| 0|
2014-01-06 00:00:00| 3| 0|
+-------------------+----------+------+

In [12]:
  .withColumn("result", F.when(F.size("productids")!=F.size(F.array_distinct("productids")),F.lit(0)).otherwise(F.col("result")))\

In [13]:
list=[['Jan/19',1],
      ['Feb/19',0],
      ['Mar/19',4],
      ['Apr/19',5],
      ['May/19',0],
      ['Jun/19',10]]

df=spark.createDataFrame(list,['Month','Value'])

df.show()

+------+-----+
 Month|Value|
+------+-----+
Jan/19| 1|
Feb/19| 0|
Mar/19| 4|
Apr/19| 5|
May/19| 0|
Jun/19| 10|
+------+-----+

In [14]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().orderBy(F.col("Month").cast("long")).rangeBetween(-(86400*89), 0)
df\
.withColumn("Month", F.to_timestamp("Month","MMM/yy"))\
.withColumn("Sum", F.sum("Value").over(w)).show()

+-------------------+-----+---+
 Month|Value|Sum|
+-------------------+-----+---+
2019-01-01 00:00:00| 1| 1|
2019-02-01 00:00:00| 0| 1|
2019-03-01 00:00:00| 4| 5|
2019-04-01 00:00:00| 5| 9|
2019-05-01 00:00:00| 0| 9|
2019-06-01 00:00:00| 10| 15|
+-------------------+-----+---+

In [15]:
w=Window().partitionBy(F.year("Month")).orderBy(F.month("Month")).rangeBetween(-2, 0)
df\
.withColumn("Month", F.to_timestamp("Month","MMM/yy"))\
.withColumn("Sum", F.sum("Value").over(w)).show()

+-------------------+-----+---+
 Month|Value|Sum|
+-------------------+-----+---+
2019-01-01 00:00:00| 1| 1|
2019-02-01 00:00:00| 0| 1|
2019-03-01 00:00:00| 4| 5|
2019-04-01 00:00:00| 5| 9|
2019-05-01 00:00:00| 0| 9|
2019-06-01 00:00:00| 10| 15|
+-------------------+-----+---+

In [16]:
list=[[{"record" : "high", "box" : "high", "tangle" : "high"}],
      [{"record" : "low", "box" : "high", "tangle" : "high"}]]
df=spark.createDataFrame(list,['record_content'])
x = ['record', 'box', 'tangle']
df.filter(' and '.join(["record_content."+i+"='high'" for i in x])).show()
df.filter(' or '.join(["record_content."+i+"='high'" for i in x])).show()

+--------------------+
 record_content|
+--------------------+
[box -> high, tan...|
+--------------------+

+--------------------+
 record_content|
+--------------------+
[box -> high, tan...|
[box -> high, tan...|
+--------------------+

In [17]:
' and '.join(["record_content."+x+"='high'" for x in new_x])

Out[69]: "record_content.record_content.record='high' and record_content.record_content.box='high' and record_content.record_content.tangle='high'"

In [18]:
' and '.join([x+"='high'" for x in new_x])

Out[66]: "record_content.record='high' and record_content.box='high' and record_content.tangle='high'"

In [19]:
' and '.join([x+"='high'" for x in new_x])

Out[62]: "col1.record='high' and col1.box='high' and col1.tangle='high'"

In [20]:
df.filter(' and '.join([x+"='high'" for x in new_x])).show()

+--------------------+
 col1|
+--------------------+
[box -> high, tan...|
+--------------------+

In [21]:

list=[[1,0.2,23.78,True,'hello_world'],
      [2,0.6,12.41,False,'byebye_world']]
  
df= spark.createDataFrame(list, ['id','model_score','tx_amount','isValid','greeting'])

df.show()
df.withColumn("id", F.col("id").cast("int")).printSchema()

+---+-----------+---------+-------+------------+
 id|model_score|tx_amount|isValid| greeting|
+---+-----------+---------+-------+------------+
 1| 0.2| 23.78| true| hello_world|
 2| 0.6| 12.41| false|byebye_world|
+---+-----------+---------+-------+------------+

root
-- id: integer (nullable = true)
-- model_score: double (nullable = true)
-- tx_amount: double (nullable = true)
-- isValid: boolean (nullable = true)
-- greeting: string (nullable = true)

In [22]:
df.withColumn("cols", F.explode(F.arrays_zip(F.array(*[F.array(x[0],F.lit(x[1]),F.lit(x[0]))\
                                                    for x in df.dtypes[1:]]))))\
        .select("id", F.col("cols.*")).withColumn("col_value", F.element_at("0",1))\
                                      .withColumn("col_type", F.element_at("0",2))\
                                      .withColumn("col_name", F.element_at("0",3)).drop("0").show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o3492.withColumn.
: org.apache.spark.sql.AnalysisException: cannot resolve 'array(`isValid`, 'boolean', 'isValid')' due to data type mismatch: input to function array should all be the same type, but it's [boolean, string, string];;
'Project [id#850L, model_score#851, tx_amount#852, isValid#853, greeting#854, explode(arrays_zip(array(array(cast(model_score#851 as string), double, model_score), array(cast(tx_amount#852 as string), double, tx_amount), array(isValid#853, boolean, isValid), array(greeting#854, string, greeting)))) AS cols#882]
+- LogicalRDD [id#850L, model_score#851, tx_amount#852, isValid#853, greeting#854], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:126)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode.org$apache$spark$sql$catalyst$trees$TreeNode$$mapChild$2(TreeNode.scala:322)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8$$anonfun$apply$13.apply(TreeNode.scala:381)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.AbstractTraversable.map(Traversable.scala:104)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:381)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode.org$apache$spark$sql$catalyst$trees$TreeNode$$mapChild$2(TreeNode.scala:322)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8$$anonfun$apply$13.apply(TreeNode.scala:381)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.AbstractTraversable.map(Traversable.scala:104

In [23]:
df.dtypes

Out[55]: [('id', 'bigint'),
 ('model_score', 'double'),
 ('tx_amount', 'double'),
 ('isValid', 'boolean'),
 ('greeting', 'string')]

In [24]:
#df.select([F.col(c).cast("string") for c in df.columns])\
df.withColumn("cols", F.explode(F.arrays_zip(F.array([F.array(x[0],F.lit(x[1]),F.lit(x[0]))\
                                                    for x in df.dtypes[1:]]))))\
        .select("id", F.col("cols.*")).show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o3492.withColumn.
: org.apache.spark.sql.AnalysisException: cannot resolve 'array(`isValid`, 'boolean', 'isValid')' due to data type mismatch: input to function array should all be the same type, but it's [boolean, string, string];;
'Project [id#850L, model_score#851, tx_amount#852, isValid#853, greeting#854, explode(arrays_zip(array(array(cast(model_score#851 as string), double, model_score), array(cast(tx_amount#852 as string), double, tx_amount), array(isValid#853, boolean, isValid), array(greeting#854, string, greeting)))) AS cols#1124]
+- LogicalRDD [id#850L, model_score#851, tx_amount#852, isValid#853, greeting#854], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:126)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode.org$apache$spark$sql$catalyst$trees$TreeNode$$mapChild$2(TreeNode.scala:322)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8$$anonfun$apply$13.apply(TreeNode.scala:381)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.AbstractTraversable.map(Traversable.scala:104)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:381)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode.org$apache$spark$sql$catalyst$trees$TreeNode$$mapChild$2(TreeNode.scala:322)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8$$anonfun$apply$13.apply(TreeNode.scala:381)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.AbstractTraversable.map(Traversable.scala:10

In [25]:
df.select([F.col(c).cast("string") for c in df.columns])\
        .withColumn("cols", F.explode(F.array(F.array([F.array(x[0],F.lit(x[1]),F.lit(x[0]))\
                                                    for x in df.dtypes[1:]]))))\
        .select("id", F.col("cols.*")).withColumn("col_value", F.element_at("0",1))\
                                      .withColumn("col_type", F.element_at("0",2))\
                                      .withColumn("col_name", F.element_at("0",3)).drop("0").show()

+---+------------+--------+-----------+
 id| col_value|col_type| col_name|
+---+------------+--------+-----------+
 1| 0.2| double|model_score|
 1| 23.78| double| tx_amount|
 1| true| boolean| isValid|
 1| hello_world| string| greeting|
 2| 0.6| double|model_score|
 2| 12.41| double| tx_amount|
 2| false| boolean| isValid|
 2|byebye_world| string| greeting|
+---+------------+--------+-----------+

In [26]:
df.withColumn("cols", F.explode(F.arrays_zip(F.array(F.array("score",F.lit("double"),F.lit("score")),\
                                                     F.array("tx_amount", F.lit("double"),F.lit("tx_amount")),\
                                                     F.array("isValid", F.lit("boolean"), F.lit("isValid")),\
                                                     F.array("greeting", F.lit("string"),F.lit("greeting"))))))\
        .select("id", F.col("cols.*")).withColumn("col_value", F.element_at("0",1))\
                                      .withColumn("col_type", F.element_at("0",2))\
                                      .withColumn("col_name", F.element_at("0",3)).drop("0").show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o2382.withColumn.
: org.apache.spark.sql.AnalysisException: cannot resolve '`score`' given input columns: [id, greeting, tx_amount, model_score, isValid];;
'Project [id#652L, model_score#653, tx_amount#654, isValid#655, greeting#656, explode(arrays_zip(array(array('score, double, score), array(cast(tx_amount#654 as string), double, tx_amount), array(isValid#655, boolean, isValid), array(greeting#656, string, greeting)))) AS cols#678]
+- LogicalRDD [id#652L, model_score#653, tx_amount#654, isValid#655, greeting#656], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:120)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode.org$apache$spark$sql$catalyst$trees$TreeNode$$mapChild$2(TreeNode.scala:322)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8$$anonfun$apply$13.apply(TreeNode.scala:381)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.AbstractTraversable.map(Traversable.scala:104)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:381)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode.org$apache$spark$sql$catalyst$trees$TreeNode$$mapChild$2(TreeNode.scala:322)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8$$anonfun$apply$13.apply(TreeNode.scala:381)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.AbstractTraversable.map(Traversable.scala:104)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:381)
	at org.apache.spark

In [27]:
df.withColumn("cols", F.explode(F.arrays_zip(F.array(F.array("score",F.lit("double"),F.lit("score")),\
                                                     F.array("id", F.lit("int"),F.lit("id")),\
                                                     F.array("tx_amount", F.lit("double"),F.lit("tx_amount")),\
                                                     F.array("isValid", F.lit("boolean"), F.lit("isValid")),\
                                                     F.array("greeting", F.lit("string"),F.lit("greeting"))))))\
        .select("id", F.col("cols.*")).withColumnRenamed("0","col1").show()

+---+--------------------+
 id| col1|
+---+--------------------+
 1|[0.2, double, score]|
 1| [1, int, id]|
 1|[23.78, double, t...|
 1|[true, boolean, i...|
 1|[hello_world, str...|
 2|[0.6, double, score]|
 2| [2, int, id]|
 2|[12.41, double, t...|
 2|[false, boolean, ...|
 2|[byebye_world, st...|
+---+--------------------+

In [28]:
list=[[2014,1.026],
       [2015,1.024],
       [2016,1.021],
       [2017,1.019],
       [2018,1.021]]

list2=[[2014,1],
       [2015,2],
       [2016,3],
       [2017,4],
       [2018,5]]
df=spark.createDataFrame(list2,['financial_year','wpi'])

df.show()


+--------------+---+
financial_year|wpi|
+--------------+---+
 2014| 1|
 2015| 2|
 2016| 3|
 2017| 4|
 2018| 5|
+--------------+---+

In [29]:
df.withColumn("cum_wpi", F.expr("aggregate(collect_list(wpi) over (order by financial_year), (acc, x) -> acc * x)")).show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o731.withColumn.
: org.apache.spark.sql.catalyst.analysis.UnresolvedException: Invalid call to dataType on unresolved object, tree: lambda 'acc
	at org.apache.spark.sql.catalyst.expressions.UnresolvedNamedLambdaVariable.dataType(higherOrderFunctions.scala:42)
	at org.apache.spark.sql.catalyst.expressions.BinaryArithmetic.dataType(arithmetic.scala:121)
	at org.apache.spark.sql.catalyst.expressions.LambdaFunction.dataType(higherOrderFunctions.scala:136)
	at org.apache.spark.sql.hive.HiveSessionCatalog$$anonfun$lookupFunction$1.apply(HiveSessionCatalog.scala:123)
	at org.apache.spark.sql.hive.HiveSessionCatalog$$anonfun$lookupFunction$1.apply(HiveSessionCatalog.scala:123)
	at scala.collection.IndexedSeqOptimized$class.prefixLengthImpl(IndexedSeqOptimized.scala:38)
	at scala.collection.IndexedSeqOptimized$class.exists(IndexedSeqOptimized.scala:46)
	at scala.collection.mutable.ArrayBuffer.exists(ArrayBuffer.scala:48)
	at org.apache.spark.sql.hive.HiveSessionCatalog.lookupFunction(HiveSessionCatalog.scala:123)
	at org.apache.spark.sql.catalyst.analysis.ResolveHigherOrderFunctions$$anonfun$apply$1$$anonfun$applyOrElse$1.apply(higherOrderFunctions.scala:39)
	at org.apache.spark.sql.catalyst.analysis.ResolveHigherOrderFunctions$$anonfun$apply$1$$anonfun$applyOrElse$1.apply(higherOrderFunctions.scala:39)
	at org.apache.spark.sql.catalyst.analysis.package$.withPosition(package.scala:53)
	at org.apache.spark.sql.catalyst.analysis.ResolveHigherOrderFunctions$$anonfun$apply$1.applyOrElse(higherOrderFunctions.scala:38)
	at org.apache.spark.sql.catalyst.analysis.ResolveHigherOrderFunctions$$anonfun$apply$1.applyOrElse(higherOrderFunctions.scala:35)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$2.apply(TreeNode.scala:279)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$2.apply(TreeNode.scala:279)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformDown(TreeNode.scala:278)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$3.apply(TreeNode.scala:284)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$3.apply(TreeNode.scala:284)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformDown(TreeNode.scala:284)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsDown$1.apply(QueryPlan.scala:84)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsDown$1.apply(QueryPlan.scala:84)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:117)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1$2.apply(QueryPlan.scala:122)
	at scala.collection.Traversabl

In [30]:
w=Window().orderBy("financial_year")
df.withColumn("arr", F.collect_list(F.col("wpi")).over(w))\
  .withColumn("arr2", F.expr("""aggregate(arr, cast(1 as bigint), (acc, x) -> cast(acc as bigint) * x)""")).show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o583.withColumn.
: org.apache.spark.sql.AnalysisException: cannot resolve 'aggregate(`arr`, CAST(1 AS BIGINT), lambdafunction((CAST(CAST(namedlambdavariable() AS BIGINT) AS DOUBLE) * namedlambdavariable()), namedlambdavariable(), namedlambdavariable()), lambdafunction(namedlambdavariable(), namedlambdavariable()))' due to data type mismatch: argument 3 requires bigint type, however, 'lambdafunction((CAST(CAST(namedlambdavariable() AS BIGINT) AS DOUBLE) * namedlambdavariable()), namedlambdavariable(), namedlambdavariable())' is of double type.;;
Project [financial_year#16L, wpi#17, arr#51, aggregate(arr#51, cast(1 as bigint), lambdafunction((cast(cast(lambda acc#56L as bigint) as double) * lambda x#57), lambda acc#56L, lambda x#57, false), lambdafunction(lambda id#58L, lambda id#58L, false)) AS arr2#55L]
+- Window [financial_year#16L, wpi#17, collect_list(wpi#17, 0, 0) windowspecdefinition(financial_year#16L ASC NULLS FIRST, specifiedwindowframe(RangeFrame, unboundedpreceding$(), currentrow$())) AS arr#51], [financial_year#16L ASC NULLS FIRST]
 +- Project [financial_year#16L, wpi#17]
 +- LogicalRDD [financial_year#16L, wpi#17], false

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:126)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$3.applyOrElse(CheckAnalysis.scala:111)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$6.apply(TreeNode.scala:303)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:302)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$5.apply(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(TreeNode.scala:300)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$transformExpressionsUp$1.apply(QueryPlan.scala:94)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$3.apply(QueryPlan.scala:106)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.transformExpression$1(QueryPlan.scala:105)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1(QueryPlan.scala:117)
	at org.apache.spark.sql.catalyst.plans.QueryPlan$$anonfun$org$apache$spark$sql$catalyst$plans$QueryPlan$$recursiveTransform$1$2.apply(QueryPlan.scala:122)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scal

In [31]:
w=Window().orderBy("financial_year")


In [32]:
import pandas as pd
import numpy as np
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf, PandasUDFType
df1=df.withColumn("cum_wpi", F.lit(1.2456))
@pandas_udf(df1.schema, PandasUDFType.GROUPED_MAP)
def grouped_map(df1):
     df1['cum_wpi']=df1['wpi'].cumprod().round(decimals=3)

     return df1
df.orderBy(F.col("financial_year").asc())\
  .groupby().apply(grouped_map).show()

+--------------+-----+-------+
financial_year| wpi|cum_wpi|
+--------------+-----+-------+
 2014|1.026| 1.026|
 2015|1.024| 1.051|
 2016|1.021| 1.073|
 2017|1.019| 1.093|
 2018|1.021| 1.116|
+--------------+-----+-------+

In [33]:
df.printSchema()

root
-- financial_year: long (nullable = true)
-- wpi: double (nullable = true)

In [34]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql import Window
@pandas_udf(DoubleType(), PandasUDFType.GROUPED_AGG)  
def cum_prod(wpi):
    return wpi.cumprod().round(decimals=2)
w = Window()\
    .partitionBy(F.lit(1))\
    .orderBy("financial_year")\
    .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
df.withColumn('cum_wpi', cum_prod(df['wpi']).over(w)).show()  

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1360489523994841> in <module> 
 8 . orderBy ( "financial_year" ) \ 
 9 . rowsBetween ( Window . unboundedPreceding , Window . unboundedFollowing ) 
 ---> 10 df . withColumn ( 'cum_wpi' , cum_prod ( df [ 'wpi' ] ) . over ( w ) ) . show ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 382 """
 383 if isinstance ( truncate , bool ) and truncate : 
 --> 384 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 385 else : 
 386 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o7659.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 18 in stage 146.0 failed 1 times, most recent failure: Lost task 18.0 in stage 146.0 (TID 416, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
 File "/databricks/spark/python/pyspark/worker.py", line 480, in main
 process()
 File "/databricks/spark/python/pyspark/worker.py", line 472, in process
 serializer.dump_stream(out_iter, outfile)
 File "/databricks/spark/python/pyspark/serializers.py", line 408, in dump_stream
 timely_flush_timeout_ms=self.timely_flush_timeout_ms)
 File "/databricks/spark/python/pyspark/serializers.py", line 215, in dump_stream
 for batch in iterator:
 File "/databricks/spark/python/pyspark/serializers.py", line 399, in init_stream_yield_batches
 batch = self._create_batch(series)
 File "/databricks/spark/python/pyspark/serializers.py", line 335, in _create_batch
 arrs.append(create_array(s, t))
 File "/databricks/spark/python/pyspark/serializers.py", line 305, in create_array
 return pa.Array.from_pandas(s, mask=mask, type=t, safe=False)
 File "pyarrow/array.pxi", line 542, in pyarrow.lib.Array.from_pandas
 File "pyarrow/array.pxi", line 169, in pyarrow.lib.array
 File "pyarrow/array.pxi", line 78, in pyarrow.lib._ndarray_to_array
 File "pyarrow/error.pxi", line 81, in pyarrow.lib.check_status
pyarrow.lib.ArrowInvalid: Could not convert 0 1.03
1 1.05
2 1.07
3 1.09
4 1.12
Name: _0, dtype: float64 with type Series: tried to convert to double

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:540)
	at org.apache.spark.sql.execution.python.ArrowPythonRunner$$anon$1.read(ArrowPythonRunner.scala:194)
	at org.apache.spark.sql.execution.python.ArrowPythonRunner$$anon$1.read(ArrowPythonRunner.scala:144)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:494)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:440)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtils$.encodeUnsafeRows(UnsafeRowBatchUtils.scala:62)
	at org.apache.spark.sql.execution.collect.Collector$$anonfun$2.apply(Collector.scala:159)
	at org.apache.spark.sql.execution.collect.Colle

In [35]:
list=[['[{"name":"name1","id":"1"}, {"name":"name2","id":"2"}]']]
#schema= StructType([MapType(StructField("name",StringType(),StringType()),
                             #StructField("id", StringType(), StringType()))])

df=spark.createDataFrame(list,['json'])
from pyspark.sql.functions import from_json, col

#json_schema = spark.read.json(df.rdd.map(lambda row: row.json)).schema
json_schema=ArrayType(StructType([ StructField("name", StringType()), StructField("id", StringType())]))
#json_schema1=StructType([(StructField(id,StringType(),True),StructField(name,StringType(),True))])
df.withColumn("yo",F.explode(F.from_json("json",json_schema))).select("yo.*").show()

+-----+---+
 name| id|
+-----+---+
name1| 1|
name2| 2|
+-----+---+

In [36]:
%scala
val df = Seq("""[{"name":"name1","id":"1"}, {"name":"name2","id":"2"}]""").toDF("mycol")

import org.apache.spark.sql.types.DataTypes._


StructType arrayItemDataType= new StructType(new structfiled("name"), new structfiled(id));
var rootType = new ArrayType(itemDataType); // 

var field = new StructField(name: "MyJson", dataType: rootType, isNullable: false);
StructType schema = new StructType(new structField(field));


df.select(from_json('mycol', schema)).show()

In [37]:
list=[['[{"name":"name1","id":"1"}, {"name":"name2","id":"2"}]']]
df=spark.createDataFrame(list,['mycol'])
df.show(truncate=False)

json_schema=ArrayType(StructType([ StructField("name", StringType()), StructField("id", StringType())]))

df.withColumn("json",F.explode(F.from_json("mycol",json_schema))).select("json.*").show()

+------------------------------------------------------+
mycol |
+------------------------------------------------------+
[{"name":"name1","id":"1"}, {"name":"name2","id":"2"}]|
+------------------------------------------------------+

+-----+---+
 name| id|
+-----+---+
name1| 1|
name2| 2|
+-----+---+

In [38]:
StructType(List(StructField(id,StringType,true),StructField(name,IntegerType,true)))

Out[49]: StructType(List(StructField(id,StringType,true),StructField(name,StringType,true)))

In [39]:
list=[['gaid','2020-03-13 08:00:00'],
     ['gaid','2020-03-18 10:00:00'],
     ['gaid','2020-03-16 02:00:00'],
     ['idfa','2020-03-17 08:00:00'],
     ['gaid','2020-03-17 01:00:00'],
     ['gaid','2020-03-18 10:00:00'],
     ['gaid','2020-03-17 01:00:00'],
     ['gaid','2020-03-18 10:00:00'],
     ['gaid','2020-03-17 13:00:00'],
     ['gaid','2020-03-18 12:00:00'],
     ['gaid','2020-03-17 20:00:00'],
     ['gaid','2020-03-17 13:00:00'],
     ['gaid','2020-03-18 12:00:00'],
     ['gaid','2020-03-17 20:00:00'],
     ['gaid','2020-03-17 13:00:00'],
     ['gaid','2020-03-18 12:00:00'],
     ['gaid','2020-03-17 20:00:00'],
     ['gaid','2020-03-13 23:00:00'],
     ['gaid','2020-03-16 05:00:00'],
     ['idfa','2020-03-16 23:00:00']]

df=spark.createDataFrame(list,['Code','hour'])
df1=df.withColumn("hour",F.to_timestamp("hour"))

In [40]:
df.withColumn("json", F.explode(F.from_json('mycol', schema))).select("json.*")

In [41]:
time_hour=[20,21,22,23,0,1,2]
df1.withColumn("range", F.arrays_overlap(F.array(F.hour("hour")),F.array(*[F.lit(x) for x in time_hour])))\
   .filter("range=true").drop("range").show()

+----+-------------------+
Code| hour|
+----+-------------------+
gaid|2020-03-16 02:00:00|
gaid|2020-03-17 01:00:00|
gaid|2020-03-17 01:00:00|
gaid|2020-03-17 20:00:00|
gaid|2020-03-17 20:00:00|
gaid|2020-03-17 20:00:00|
gaid|2020-03-13 23:00:00|
idfa|2020-03-16 23:00:00|
+----+-------------------+

In [42]:
time_hour=[20,21,22,23,0,1,2]
df1.filter("hour(hour) >=20 or hour(hour)<=2").show()

+----+-------------------+
Code| hour|
+----+-------------------+
gaid|2020-03-16 02:00:00|
gaid|2020-03-17 01:00:00|
gaid|2020-03-17 01:00:00|
gaid|2020-03-17 20:00:00|
gaid|2020-03-17 20:00:00|
gaid|2020-03-17 20:00:00|
gaid|2020-03-13 23:00:00|
idfa|2020-03-16 23:00:00|
+----+-------------------+

In [43]:
list=   [[['rand one','rand two']],
         [['rand four','rand five']]]

df=spark.createDataFrame(list,['col1'])
df.show()

+--------------------+
 col1|
+--------------------+
[rand one, rand two]|
[rand four, rand ...|
+--------------------+

In [44]:
df.withColumn("col2", F.expr("""transform(col1, x-> regexp_replace(x,'rand ',''))""")).show()

+--------------------+------------+
 col1| col2|
+--------------------+------------+
[rand one, rand two]| [one, two]|
[rand four, rand ...|[four, five]|
+--------------------+------------+

In [45]:
df.show()

+--------------------+
 col1|
+--------------------+
[rand one, rand two]|
[rand four, rand ...|
+--------------------+

In [46]:
x = [('1', 'hello'),('2','Hi'),('3', 'Hello')]


df.withColumn("col1",F.array([F.struct(F.lit((str(i[0])+','+str(i[1])))) for i in x])).show(truncate=False)

+------------------------------+
col1 |
+------------------------------+
[[1,hello], [2,Hi], [3,Hello]]|
[[1,hello], [2,Hi], [3,Hello]]|
+------------------------------+

In [47]:
x = [('1', 'hello'),('2','Hi'),('3', 'Hello')]
df.withColumn("col1",F.array(*[F.create_map(F.lit(i[0]),F.lit(i[1])) for i in x])).show(truncate=False)

+---------------------------------------+
col1 |
+---------------------------------------+
[[1 -> hello], [2 -> Hi], [3 -> Hello]]|
[[1 -> hello], [2 -> Hi], [3 -> Hello]]|
+---------------------------------------+

In [48]:
x = [('1', 'hello'),('2','Hi'),('3', 'Hello')]
df.withColumn("col1", F.array(*[F.struct(F.lit(i[0]),F.lit(i[1])) for i in x])).show(truncate=False)

+---------------------------------+
col1 |
+---------------------------------+
[[1, hello], [2, Hi], [3, Hello]]|
[[1, hello], [2, Hi], [3, Hello]]|
+---------------------------------+

In [49]:
x = [('1', 'hello'),('2','Hi'),('3', 'Hello')]
df.withColumn("col1", F.array(*[F.array(F.lit(i[0]),F.lit(i[1])) for i in x])).show(truncate=False)

+---------------------------------+
col1 |
+---------------------------------+
[[1, hello], [2, Hi], [3, Hello]]|
[[1, hello], [2, Hi], [3, Hello]]|
+---------------------------------+